<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/master/doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preview of CNN based on bee_classification
-------------------

## Spis treści
1. [Wstęp](#1)
    1. [Cel projektu](#1.1)
    2. [Czym jest CNN?](#1.2)
    3. [Z czego składa się bee_dataset?](#1.3)
2. [Model](#2)
    1. [Specyfika modelu](#2.1)
    2. [Część eksperymentalna](#2.2)
        1. [Architektura](#2.2.1)
        2. [Porównanie jakości klasyfikacji](#2.2.2)
    3. [Użytkowanie](#2.3)
        1. [Przygotowanie danych](#2.3.1)
        2. [Opis input-output](#2.3.2)
3. [Contributing](#3)
4. [Źródła](#4)


<a name="1"></a>
# Wstęp
-------------------

<a name="1.1"></a>
## Cel projektu
-------------------
Projekt ma na celu przedstawienie funkcjonowania konwolucyjnych sieci neuronowych przy pomocy Keras oraz podstawowej bazy danych pakietu tensorflow_datasets.

<a name="1.2"></a>
## Czym jest CNN?
-------------------
Aby odpowiedzieć na pytanie, czym jest **Konwolucyjna Sieć Neuronowa** (ang. Convolutional Neural Network, inaczej CNN), pierw musimy przybliżyć pojęcie sztucznej sieci neuronowej.<br/>
Sieci neuronowe są w ogólności modelem obliczeniowym inspirowanym strukturą i funkcją ludzkiego mózgu. Składają się z dużej liczby prostych jednostek przetwarzających, zwanych sztucznymi neuronami, które są zorganizowane w warstwy i połączone ze sobą za pomocą wag i odchyleń, by jak najlepiej rozpoznać podstawowe relacje w zbiorze danych.

Rozłóżmy sieć neuronową na jej podstawowe bloki konstrukcyjne:
1. **Tensor** jest uogólnieniem macierzy na dowolną liczbę wymiarów. Są one używane do reprezentowania danych w sieci neuronowej - prawie zawsze są to dane liczbowe.
![tensor](https://drive.google.com/uc?id=1ZvNacNdu_RG0jr5U2PrwVoQsRg07Jzjb)

2. **Neuron** można przedstawić jako funkcję, która przyjmuje wiele wejść i daje jedno wyjście.
![sztuczny neuron](https://drive.google.com/uc?id=1k3OCAmfXWrhs_lv-ER75e0I9OOaUqaDQ)

3. **Warstwa** to zbiór neuronów o tym samym działaniu, z tymi samymi hiperparametrami.

4. **Wagi filtra** (inaczej wagi jądra lub maski), choć unikalne dla każdego neuronu, są dostrajane w fazie treningu w celu optymalizacji wydajności sieci na danym zadaniu.

5. **Odchylenie** (ang. bias) jest skalarem, który pozwala budować model liniowy, który nie jest ustalony na początku i ma możliwość przeszukiwania przestrzeni rozwiązań. Odchylenie jest uczone wraz z wagami filtra.
![sztuczny neuron](https://drive.google.com/uc?id=1_XR0_czVA84L2-GpoVlHluIfKWvuYN_9)

**Czym różni się CNN od klasycznego modelu uczenia maszynowego?**

CNN wykorzystuje specjalny typ warstwy - warstwę konwolucyjną, która jest dobrze przygotowana do uczenia się z obrazów i danych obrazopodobnych. Sieć taka może być wykorzystana do zadań związanych z widzeniem komputerowym, takich jak przetwarzanie obrazów, klasyfikacja, segmentacja i wykrywanie obiektów.<br/><br/>

**Czy zajmują sie poszczególne warstwy sieci?**

**Warstwa wejściowa** (ang. Input Layer) - pierwsza warstwa sieci, ma za zadanie przechwycenie sygnału i przekazanie go do kolejnych warstw sieci (zwanych warstwami ukrytymi).

**Warstwa konwolucyjna** (ang. Convolutional Layer) - stosuje filtry do danych wejściowych i uczy się rozpoznawania cech. Początkowe warstwy konwolucyjne wydobywają ogólne lub niskopoziomowe cechy, takie jak linie i krawędzie, a kolejne warstwy uczą się subtelniejszych szczegółów lub cech wysokopoziomowych. 
![warstwa konwolucyjna](https://drive.google.com/uc?id=1-qrBQ0ynVIu5X8Nr9ZaQmtFX_OKJeD6j)

**Funkcje aktywacyjne** (ang. Activation Functions) - nakładane elementarnie na dane wejściowe po nałożeniu wag. Wprowadzają one nieliniowość do sieci, pozwalając modelowi na uczenie się bardziej złożonych funkcji. Niektóre z najbardziej popularnych funkcji aktywacji obejmują:
![sztuczny neuron](https://drive.google.com/uc?id=1E23OOrObsAkbnJ27MG_pRQnDNZnB2sA6)

**Warstwa łącząca** (ang. Pooling Layer) - jest używana do zmniejszania rozmiaru objętości wejściowej, aby zmniejszyć moc obliczeniową wymaganą do przetworzenia obrazu, nie tracąc przy tym właściwości obrazu.

**Warstwa dropout** (ang. Dropout Layer) - losowo wyrzuca pewne neurony podczas treningu by zapobiec nadmiernemu dopasowaniu.

**Warstwa w pełni połączona** (ang. Fully Connected Layer albo Dense Layer) pobiera wyjście z końcowej warstwy łączącej i wykorzystują je do tworzenia predykcji przy użyciu standardowych technik sieci neuronowych. Jest zazwyczaj ostatnią warstwą przed warstwą wyjściową.

**Funkcja straty** (ang. Loss Function) - pobiera ostateczne wyjście sieci, wytworzone przez warstwę w pełni połączoną, i porównuje je z wyjściem prawdziwym. Na tej podstawie obliczany jest błąd który jest wykorzystywany do 
aktualizacji wag i odchyleń.

**Warstwa wyjściowa** (ang. Output Layer) - ostatnia warstw sieci, w której neuronach formowane są sygnały wyjściowe.<br/><br/>

**Co może doprowadzić do nadmiernego dopasowanie danych?**

**Nadmierne Dopasowanie** (ang. Overfitting) to zjawisko, które występuje, gdy model jest zbyt złożony i jest w stanie dopasować się do szumu lub losowych zmian w danych treningowych, a nie do leżących u jego podstaw wzorców.<br/>
Istnieje kilka czynników, które mogą do tego doprowadzić:
  * Niewystarczająca ilość danych

  * Duża złożoność modelu: Model o zbyt wielu parametrach lub wysokim stopniu swobody.

  * Brak regularności: Regularność jest techniką dodającą termin kary do funkcji straty, która zniechęca do dużych wag, zapobiegając przy tym dopasowaniu do szumu w danych treningowych.

  * Wyciek danych: Wyciek danych występuje, gdy informacje z zestawu testowego są używane do informowania modelu podczas szkolenia. Model będzie uzyskiwał dobre wyniki na zestawie testowym, ale nie będzie dobrze generalizować na nowe przykłady.

<a name="1.3"></a>
## Z czego składa się bee_dataset?
-------------------

[bee_dataset](https://www.tensorflow.org/datasets/catalog/bee_dataset)

czym jest dataset
opis+zdj
format(train+test, rozmiar, etykiety)

**Czym jest przetwarzanie danych?**
...

Zaliczają się do nich:<br/>
  **Noramlizacja** (ang. Normalisation) - najważniejszy krok w części wstępnego przetwarzania. Polega ona na przeskalowaniu wartości pikseli tak, aby mieściły się one w określonym zakresie(najczęściej 0-1). Umożliwia wyższe prędkości uczenia się i zmniejszenie silnej zależności od inicjalizacji.

  **Standaryzacja** (ang. Standardization) - przekształcenia danych w taki sposób, aby średnia danych była równa 0, a odchylenie standardowe 1.

  **Kodowanie etykiety** (ang. Label Encoding) - reprezentowania zmiennych kategorycznych jako wektor wartości o wartości 0 lub 1.

  **Przekształcenia morfologiczne obrazów** (ang. Morphological Image Transformations) - modyfikacje obejmujące kształt i formę obrazów. Zaliczają się do nich:
  * Tworzenie obrazu binarnego
  * Erozja (kurczenie jasnych regionów i powiększanie ciemnych)
  * Dylatacja (kurczenie ciemnych regionów i powiększanie jasnych)
  * Otwarcie (dylatacja poprzedzona erozją, proces usuwa małe jasne plamy tzw. sól)
  * Zamknięcie (erozja poprzedzona dylatacją, proces usuwa małe czarne plamy tzw. pieprz)

**Rozszerzenie danych** (ang. Data Augmentation) - zestaw technik, które sztucznie generują nowe dane z istniejących danych. Zaliczają się do nich: obrót, rotacja, losowe wycinki, zmiana koloru, dodanie szumu, utrata informacji, zmiana kontrastu.<br/><br/>

**W jaki sposób przetwarzamy naszą bazę danych?**
gdzie czemu została poddana 

<a name="2"></a>
# Model
-------------------

<a name="2.1"></a>
## Specyfika modelu
-------------------

<a name="2.2"></a>
## Część eksperymentalna
-------------------

<a name="2.2.1"></a>
### Architektura
-------------------

<a name="2.2.2"></a>
### Porównanie jakości klasyfikacji
-------------------

<a name="2.3"></a>
## Użytkowanie
-------------------

<a name="2.3.1"></a>
### Przygotowanie danych
-------------------

<a name="2.3.2"></a>
### Opis input-output
-------------------

<a name="3"></a>
# Contributing
-------------------

<a name="4"></a>
# Źródła
-------------------
1. [Deep Learning with Python by François Chollet, version 6, 2017](https://www.manning.com/books/deep-learning-with-python)

2. [Hands-On Machine Learning with Scikit-Learn, Keras and TensorFlow by Aurélien Géron, 2nd edition, 2019](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

3. [TensorFlow Keras Documentation](https://www.tensorflow.org/api_docs/python/tf/keras)

4. [CNN Explainer](https://github.com/poloclub/cnn-explainer)

5. [Deep Learning cheatsheets for Stanford's CS 230](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-deep-learning-tips-and-tricks)

6. [Jaki jest związek pomiędzy sieciami CNN i głębokim uczeniem?](https://www.intel.pl/content/www/pl/pl/internet-of-things/computer-vision/convolutional-neural-networks.html)

7. Wykład "Uczenie maszynowe w Pythonie" dr Macieja Ślęczki

8. [Wikipedia](http://wikipedia.org)


<br/>

![zdjecie](https://drive.google.com/uc?id=16xYDkCsPLDC8VhVVwB5bq3rpxdC-fnYM)
//note: "https://drive.google.com/uc?id=" link format

https://playground.tensorflow.org/

<style type="text/css">
    ol { list-style-type: decimal; }
</style>